# Sarcasm Detection
 **Acknowledgement**

Misra, Rishabh, and Prahal Arora. "Sarcasm Detection using Hybrid Neural Network." arXiv preprint arXiv:1908.07414 (2019).

**Required Files given in below link.**

https://drive.google.com/drive/folders/1xUnF35naPGU63xwRDVGc-DkZ3M8V5mMk

Good Link to refer the Keras.Embedding Example

https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

## Install `Tensorflow2.0` 

In [ ]:
!!pip uninstall tensorflow
!pip install tensorflow==2.0.0

     |████████████████████████████████| 86.3MB 34kB/s 
     |████████████████████████████████| 450kB 53.6MB/s 
     |████████████████████████████████| 51kB 9.1MB/s 
     |████████████████████████████████| 3.8MB 55.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=c599d11cc8057b9819e1e9c7df08bfe55e0ca18ae21d683e3681e6d1d7edbe33
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstal

## Get Required Files from Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
#Set your project path 
project_path =  "/content/drive/My Drive/Sequence Modelling_OnlineSession_Krishna/Project_SequenceModel/"## Add your path here ##""

In [4]:
import pandas as pd
import numpy as np

#**## Reading and Exploring Data**

## Read Data "Sarcasm_Headlines_Dataset.json". Explore the data and get  some insights about the data. ( 4 marks)
Hint - As its in json format you need to use pandas.read_json function. Give paraemeter lines = True.

In [5]:
pwd

'/content'

In [6]:
df = pd.read_json(project_path+'Sarcasm_Headlines_Dataset.json', lines= True)

In [7]:
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


## Drop `article_link` from dataset. ( 2 marks)
As we only need headline text data and is_sarcastic column for this project. We can drop artical link column here.

In [8]:
df.drop('article_link', axis = 1, inplace = True)
df.head()

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


## Get the Length of each line and find the maximum length. ( 4 marks)
As different lines are of different length. We need to pad the our sequences using the max length.

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26709 entries, 0 to 26708
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   headline      26709 non-null  object
 1   is_sarcastic  26709 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 417.5+ KB


In [10]:
len(df['headline'][0].split(" "))

12

In [11]:
df['len'] = df['headline'].apply(lambda x: len(x.split(" ")))

In [12]:
df.head()

,headline,is_sarcastic,len
0,former versace store clerk sues over secret 'b...,0,12
1,the 'roseanne' revival catches up to our thorn...,0,14
2,mom starting to fear son's web series closest ...,1,14
3,"boehner just wants wife to listen, not come up...",1,13
4,j.k. rowling wishes snape happy birthday in th...,0,11


In [13]:
df['headline'][0]

"former versace store clerk sues over secret 'black code' for minority shoppers"

In [14]:
# word = "Hello how are you,"
# print(word.split(" ")), print(len(word.split(" ")))

In [15]:
# Length = []
# for i in range(len(df)):
#   Length.append(len(df['headline'][i]))

# print(Length)

In [16]:
np.max(df['len']), np.argmax(df['len'])

(39, 15247)

In [17]:
df['headline'][15247]

'elmore leonard, modern prose master, noted for his terse prose style and for writing about things perfectly and succinctly with a remarkable economy of words, unfortunately and sadly expired this gloomy tuesday at the age of 87 years old'

elmore leonard, modern prose master, noted for his terse prose style and for writing about things perfectly and succinctly with a remarkable economy of words, unfortunately and sadly expired this gloomy tuesday at the age of 87 years old

#**## Modelling**

## Import required modules required for modelling.

In [18]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential

# Set Different Parameters for the model. ( 2 marks)

In [19]:
max_features = 10000
maxlen = 39 ## Add your max length here ##
embedding_size = 200

In [20]:
#Sample for Understandin the function
word = "I am good now fine fine good now"
print(word)
t = Tokenizer(num_words=4)
t.fit_on_texts([word])
print(t.word_index)
print(len(t.word_counts))
test = ["I fine now"]
print(t.texts_to_sequences(test))



I am good now fine fine good now
{'good': 1, 'now': 2, 'fine': 3, 'i': 4, 'am': 5}
5
[[3, 2]]


## Apply Keras Tokenizer of headline column of your data.  ( 4 marks)
Hint - First create a tokenizer instance using Tokenizer(num_words=max_features) 
And then fit this tokenizer instance on your data column df['headline'] using .fit_on_texts()

In [21]:
# Here max_features denotes the no of most frequent words need to refer. Here in the example although total unique words are 29656
# But we shall consider most frequent 10000 words for model.
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df['headline'])

In [22]:
len(tokenizer.word_counts), len(tokenizer.word_index)

(29656, 29656)

In [23]:
#Sample for Understandin the function
test = ['to of the people out']
test1 = tokenizer.texts_to_sequences(test)
print(test1)
print(pad_sequences(test1, maxlen = 7))

[[1, 2, 3, 66, 22]]
[[ 0  0  1  2  3 66 22]]


# Define X and y for your model.

In [24]:
X = tokenizer.texts_to_sequences(df['headline'])

In [25]:
# Checking the length of a record after tokenization.
# The length has reducded from 39 to 32 because some words are dropped which doesnt come under first most frequent 10000. 
len(X[15247]), df['len'][15247]

(32, 39)

In [26]:
# Lets check which all words are dropped for this record.
df['headline'][15247] # Actual Record

'elmore leonard, modern prose master, noted for his terse prose style and for writing about things perfectly and succinctly with a remarkable economy of words, unfortunately and sadly expired this gloomy tuesday at the age of 87 years old'

In [27]:
tokenizer.sequences_to_texts([X[15247]]) # After Tokenization

['leonard modern master noted for his style and for writing about things perfectly and with a remarkable economy of words unfortunately and sadly this tuesday at the age of 87 years old']

In [28]:
tokenizer.word_index["elmore"] # Index of this dropped word is > than 10000.

23423

In [29]:
# X = tokenizer.texts_to_sequences(df['headline'])

X = pad_sequences(X, maxlen = maxlen)
y = df['is_sarcastic']
y1 = np.asarray(df['is_sarcastic'])

print("Number of Samples:", len(X))
print(X[15247])
print("Number of Labels: ", len(y))
print(y[15247])

Number of Samples: 26709
[   0    0    0    0    0    0    0 7744 1759 3334 8865    5   31  811
    8    5 1316   16  134 1047    8    9    6 6272  932    2  716 6073
    8 3279   18 3218   15    3  483    2 3051  102   60]
Number of Labels:  26709
1


In [30]:
y1,y

(array([0, 0, 1, ..., 0, 0, 0]), 0        0
 1        0
 2        1
 3        1
 4        0
         ..
 26704    0
 26705    0
 26706    0
 26707    0
 26708    0
 Name: is_sarcastic, Length: 26709, dtype: int64)

## Get the Vocabulary size ( 2 marks)
Hint : You can use tokenizer.word_index.

In [31]:
tokenizer.num_words

10000

In [32]:
len(tokenizer.word_index)

29656

In [33]:
tokenizer.word_index

{'to': 1,
 'of': 2,
 'the': 3,
 'in': 4,
 'for': 5,
 'a': 6,
 'on': 7,
 'and': 8,
 'with': 9,
 'is': 10,
 'new': 11,
 'trump': 12,
 'man': 13,
 'from': 14,
 'at': 15,
 'about': 16,
 'you': 17,
 'this': 18,
 'by': 19,
 'after': 20,
 'up': 21,
 'out': 22,
 'be': 23,
 'how': 24,
 'as': 25,
 'it': 26,
 'that': 27,
 'not': 28,
 'are': 29,
 'your': 30,
 'his': 31,
 'what': 32,
 'he': 33,
 'all': 34,
 'just': 35,
 'who': 36,
 'has': 37,
 'will': 38,
 'more': 39,
 'one': 40,
 'into': 41,
 'report': 42,
 'year': 43,
 'why': 44,
 'have': 45,
 'area': 46,
 'over': 47,
 'donald': 48,
 'u': 49,
 'day': 50,
 'says': 51,
 's': 52,
 'can': 53,
 'first': 54,
 'woman': 55,
 'time': 56,
 'like': 57,
 'her': 58,
 "trump's": 59,
 'old': 60,
 'no': 61,
 'get': 62,
 'off': 63,
 'an': 64,
 'life': 65,
 'people': 66,
 'obama': 67,
 'now': 68,
 'house': 69,
 'still': 70,
 "'": 71,
 'women': 72,
 'make': 73,
 'was': 74,
 'than': 75,
 'white': 76,
 'back': 77,
 'my': 78,
 'i': 79,
 'clinton': 80,
 'down': 81,
 'i

#**## Word Embedding**

## Get Glove Word Embeddings

In [ ]:
# glove_file = project_path + "glove.6B.zip"

In [ ]:
# #Extract Glove embedding zip file
# from zipfile import ZipFile
# with ZipFile(glove_file, 'r') as z:
#   z.extractall()

# Get the Word Embeddings using Embedding file as given below.

In [34]:
EMBEDDING_FILE = project_path+'glove.6B.200d.txt'

embeddings = {}
for o in open(EMBEDDING_FILE):
    word = o.split(" ")[0]
    # print(word)
    embd = o.split(" ")[1:]
    embd = np.asarray(embd, dtype='float32')
    # print(embd)
    embeddings[word] = embd

In [35]:
len(embeddings.keys()), len(embeddings.values())

(400000, 400000)

In [36]:
len(embeddings['the'])

200

In [37]:
embeddings['the']

array([-7.1549e-02,  9.3459e-02,  2.3738e-02, -9.0339e-02,  5.6123e-02,
        3.2547e-01, -3.9796e-01, -9.2139e-02,  6.1181e-02, -1.8950e-01,
        1.3061e-01,  1.4349e-01,  1.1479e-02,  3.8158e-01,  5.4030e-01,
       -1.4088e-01,  2.4315e-01,  2.3036e-01, -5.5339e-01,  4.8154e-02,
        4.5662e-01,  3.2338e+00,  2.0199e-02,  4.9019e-02, -1.4132e-02,
        7.6017e-02, -1.1527e-01,  2.0060e-01, -7.7657e-02,  2.4328e-01,
        1.6368e-01, -3.4118e-01, -6.6070e-02,  1.0152e-01,  3.8232e-02,
       -1.7668e-01, -8.8153e-01, -3.3895e-01, -3.5481e-02, -5.5095e-01,
       -1.6899e-02, -4.3982e-01,  3.9004e-02,  4.0447e-01, -2.5880e-01,
        6.4594e-01,  2.6641e-01,  2.8009e-01, -2.4625e-02,  6.3302e-01,
       -3.1700e-01,  1.0271e-01,  3.0886e-01,  9.7792e-02, -3.8227e-01,
        8.6552e-02,  4.7075e-02,  2.3511e-01, -3.2127e-01, -2.8538e-01,
        1.6670e-01, -4.9707e-03, -6.2714e-01, -2.4904e-01,  2.9713e-01,
        1.4379e-01, -1.2325e-01, -5.8178e-02, -1.0290e-03, -8.21

In [ ]:
# EMBEDDING_FILE = './glove.6B.200d.txt'

# embeddings = {}
# for o in open(EMBEDDING_FILE):
#     word = o.split(" ")[0]
#     # print(word)
#     embd = o.split(" ")[1:]
#     embd = np.asarray(embd, dtype='float32')
#     # print(embd)
#     embeddings[word] = embd



# Create a weight matrix for words in training docs

In [43]:
num_words = len(tokenizer.word_index)+1  # +1 is done to taken into account the padding char.
embedding_matrix = np.zeros((num_words, 200))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

len(embeddings.values()), num_words

(400000, 29657)

In [44]:
embedding_matrix[3], embeddings['the']

(array([-7.15489984e-02,  9.34590027e-02,  2.37380005e-02, -9.03389975e-02,
         5.61229996e-02,  3.25470001e-01, -3.97960007e-01, -9.21389982e-02,
         6.11810014e-02, -1.89500004e-01,  1.30610004e-01,  1.43490002e-01,
         1.14789996e-02,  3.81579995e-01,  5.40300012e-01, -1.40880004e-01,
         2.43149996e-01,  2.30360001e-01, -5.53390026e-01,  4.81540002e-02,
         4.56620008e-01,  3.23379993e+00,  2.01990008e-02,  4.90190014e-02,
        -1.41319996e-02,  7.60169998e-02, -1.15270004e-01,  2.00599998e-01,
        -7.76569992e-02,  2.43279994e-01,  1.63680002e-01, -3.41179997e-01,
        -6.60699978e-02,  1.01520002e-01,  3.82319987e-02, -1.76679999e-01,
        -8.81529987e-01, -3.38950008e-01, -3.54809985e-02, -5.50949991e-01,
        -1.68990009e-02, -4.39819992e-01,  3.90040018e-02,  4.04469997e-01,
        -2.58800000e-01,  6.45940006e-01,  2.66409993e-01,  2.80090004e-01,
        -2.46249996e-02,  6.33019984e-01, -3.17000002e-01,  1.02710001e-01,
         3.0

In [45]:
embedding_vector

array([ 0.36962  , -0.040156 , -0.023737 , -0.31948  ,  0.34988  ,
       -0.24554  , -0.12108  ,  0.51612  , -0.074941 ,  0.2204   ,
       -0.05352  ,  0.35542  ,  0.99586  ,  0.1107   , -0.26814  ,
        0.19858  ,  0.01581  ,  0.11829  , -0.33452  , -0.25706  ,
       -0.014353 , -0.25862  ,  0.41545  ,  0.07123  ,  0.1366   ,
        0.022458 ,  0.65286  , -0.43146  , -0.24555  , -0.10398  ,
       -0.30657  ,  0.59368  ,  0.34855  ,  0.43896  ,  0.34273  ,
        0.22965  ,  0.063959 ,  0.4975   ,  0.081507 ,  0.27792  ,
        0.32306  ,  0.41494  , -0.3137   ,  0.29781  , -0.24366  ,
       -0.048078 ,  0.082093 ,  0.57941  ,  0.44228  ,  0.067831 ,
       -0.18277  , -0.14844  ,  0.20495  ,  0.44198  , -0.15826  ,
        0.081587 ,  0.041577 , -0.47717  , -0.31472  ,  0.33353  ,
       -0.19855  , -0.023786 , -0.47119  , -0.0058474, -0.41529  ,
        0.030389 ,  0.10989  ,  0.057665 ,  0.29621  , -0.0045144,
       -0.079666 ,  0.18944  , -0.15832  , -0.34246  , -0.0160

In [49]:
embedding_matrix[29656]

array([ 0.36962   , -0.040156  , -0.023737  , -0.31948   ,  0.34988001,
       -0.24553999, -0.12108   ,  0.51612002, -0.074941  ,  0.22040001,
       -0.05352   ,  0.35541999,  0.99585998,  0.1107    , -0.26813999,
        0.19858   ,  0.01581   ,  0.11829   , -0.33452001, -0.25705999,
       -0.014353  , -0.25861999,  0.41545001,  0.07123   ,  0.1366    ,
        0.022458  ,  0.65285999, -0.43145999, -0.24555001, -0.10398   ,
       -0.30656999,  0.59368002,  0.34854999,  0.43895999,  0.34272999,
        0.22965001,  0.063959  ,  0.4975    ,  0.081507  ,  0.27792001,
        0.32306001,  0.41494   , -0.31369999,  0.29780999, -0.24366   ,
       -0.048078  ,  0.082093  ,  0.57941002,  0.44227999,  0.067831  ,
       -0.18277   , -0.14844   ,  0.20495   ,  0.44198   , -0.15826   ,
        0.081587  ,  0.041577  , -0.47716999, -0.31472   ,  0.33353001,
       -0.19855   , -0.023786  , -0.47119001, -0.0058474 , -0.41529   ,
        0.030389  ,  0.10989   ,  0.057665  ,  0.29620999, -0.00

## Create and Compile your Model  ( 7 marks)
Hint - Use Sequential model instance and then add Embedding layer, Bidirectional(LSTM) layer, then dense and dropout layers as required. 
In the end add a final dense layer with sigmoid activation for binary classification.


In [ ]:
### Embedding layer for hint 
## model.add(Embedding(num_words, embedding_size, weights = [embedding_matrix]))
### Bidirectional LSTM layer for hint 
## model.add(Bidirectional(LSTM(128, return_sequences = True)))

In [56]:
model = Sequential()
model.add(Embedding(num_words, embedding_size, weights = [embedding_matrix], trainable = False))
model.add(Bidirectional(LSTM(128, return_sequences = True)))
model.add(Dropout(0.4))
model.add(Dense(80, activation= 'relu'))
model.add(Dropout(0.4))
model.add(Dense(40, activation= 'relu'))
model.add(Dropout(0.4))
model.add(Dense(20, activation= 'relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation= 'sigmoid'))
model.compile(optimizer= 'adam', loss = 'binary_crossentropy', metrics= ['accuracy'])

In [57]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 200)         5931400   
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 256)         336896    
_________________________________________________________________
dropout_6 (Dropout)          (None, None, 256)         0         
_________________________________________________________________
dense_6 (Dense)              (None, None, 80)          20560     
_________________________________________________________________
dropout_7 (Dropout)          (None, None, 80)          0         
_________________________________________________________________
dense_7 (Dense)              (None, None, 40)          3240      
_________________________________________________________________
dropout_8 (Dropout)          (None, None, 40)         

# Fit your model with a batch size of 100 and validation_split = 0.2. and state the validation accuracy ( 5 marks)


In [70]:
batch_size = 100
epochs = 10

model.fit(X,y1,batch_size= batch_size, epochs= epochs, validation_split = 0.2 )
## Add your code here ##

Epoch 1/10
214/214 [==============================] - 3s 13ms/step - loss: 0.0915 - accuracy: 0.9672 - val_loss: 0.5893 - val_accuracy: 0.8518
Epoch 2/10
214/214 [==============================] - 3s 13ms/step - loss: 0.0915 - accuracy: 0.9677 - val_loss: 0.7516 - val_accuracy: 0.8550
Epoch 3/10
214/214 [==============================] - 3s 13ms/step - loss: 0.0694 - accuracy: 0.9757 - val_loss: 0.9575 - val_accuracy: 0.8525
Epoch 4/10
214/214 [==============================] - 3s 13ms/step - loss: 0.0681 - accuracy: 0.9768 - val_loss: 0.7423 - val_accuracy: 0.8556
Epoch 5/10
214/214 [==============================] - 3s 13ms/step - loss: 0.0516 - accuracy: 0.9831 - val_loss: 0.8334 - val_accuracy: 0.8563
Epoch 6/10
214/214 [==============================] - 3s 13ms/step - loss: 0.0455 - accuracy: 0.9847 - val_loss: 0.9274 - val_accuracy: 0.8487
Epoch 7/10
214/214 [==============================] - 3s 13ms/step - loss: 0.0420 - accuracy: 0.9863 - val_loss: 0.8786 - val_accuracy: 0.8541

In [72]:
y1[1]

0

## Test the Model

#### Question for Evaluator - I have query here, I am getting 39 results for the class prediction, Although the lables are eiter 0,1. How to get one result for the prediction class. Can you please advice it in evaluation comments. Thanks

In [71]:

X[1].shape , model.predict(X[1]).shape

((39,), (39, 1, 1))

In [66]:
print(model.predict_classes(X[1]))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
[[[1]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[0]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]

 [[1]]

 [[0]]

 [[0]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]

 [[0]]]


In [65]:
print(X[1])
print(model.predict(X[1]))

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    3 8434 3337
 2745   21    1  165 8435  415 3111    5  257    8 1001]
[[[0.5914271 ]]

 [[0.5914271 ]]

 [[0.5914271 ]]

 [[0.5914271 ]]

 [[0.5914271 ]]

 [[0.5914271 ]]

 [[0.5914271 ]]

 [[0.5914271 ]]

 [[0.5914271 ]]

 [[0.5914271 ]]

 [[0.5914271 ]]

 [[0.5914271 ]]

 [[0.5914271 ]]

 [[0.5914271 ]]

 [[0.5914271 ]]

 [[0.5914271 ]]

 [[0.5914271 ]]

 [[0.5914271 ]]

 [[0.5914271 ]]

 [[0.5914271 ]]

 [[0.5914271 ]]

 [[0.5914271 ]]

 [[0.5914271 ]]

 [[0.5914271 ]]

 [[0.5914271 ]]

 [[0.01083101]]

 [[0.5914271 ]]

 [[0.38859305]]

 [[0.17280822]]

 [[0.4103815 ]]

 [[0.649072  ]]

 [[0.15436342]]

 [[0.4715953 ]]

 [[0.51326996]]

 [[0.7599343 ]]

 [[0.45664445]]

 [[0.21583831]]

 [[0.00953017]]

 [[0.3416713 ]]]


In [64]:
df.headline[1], y[1]

("the 'roseanne' revival catches up to our thorny political mood, for better and worse",
 0)

In [ ]:
# sample = tokenizer.texts_to_sequences('What a great time')
# sample = pad_sequences(sample, maxlen = maxlen)

# k = model.predict(sample)
# k

array([[[0.26293433],
        [0.26521695],
        [0.26385933],
        [0.25934157],
        [0.25335953],
        [0.24853745],
        [0.24718247],
        [0.2506279 ],
        [0.25834152],
        [0.26818028],
        [0.27742964],
        [0.28279537],
        [0.28173003],
        [0.27301732],
        [0.259037  ],
        [0.24320668],
        [0.22908407],
        [0.21920878],
        [0.21453784],
        [0.2143703 ],
        [0.21651727],
        [0.21778944],
        [0.21503365],
        [0.20653461],
        [0.19284134],
        [0.17684394],
        [0.16215065],
        [0.15134408],
        [0.14537667],
        [0.14410241],
        [0.14590357],
        [0.14788707],
        [0.14785215],
        [0.14457062],
        [0.13952118],
        [0.135415  ],
        [0.13656081],
        [0.14885694],
        [0.09767932]],

       [[0.81551397],
        [0.812207  ],
        [0.8070126 ],
        [0.8006335 ],
        [0.79458284],
        [0.79067767],
        